In [1]:
"""
将模型作为层使用（Model as Layer）

在函数式API中，Model本身可以像Layer一样被调用和复用。
这种技术常用于：
1. 特征提取器复用：在多个分支上使用相同的预训练模型
2. 复杂架构构建：将子网络封装为模块化组件
3. 迁移学习：冻结预训练模型权重，仅训练新增层

本示例：构建双目视觉模型（Stereo Vision）
使用Xception作为共享特征提取器，处理左右两个视角的图像
"""

from keras import layers
from keras import applications
from keras.layers import Input
from keras.models import Model
import numpy as np

# 设置随机种子
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

# 加载Xception模型作为特征提取器
# weights=None：从头训练（实际应用中通常使用'imagenet'预训练权重）
# include_top=False：移除顶层分类器，仅保留卷积基
xception_base = applications.Xception(weights=None, include_top=False)

# 左视角输入
left_input = Input(shape=(250, 250, 3), name='left_camera')
left_features = xception_base(left_input)

# 右视角输入，复用相同的Xception模型
right_input = Input(shape=(250, 250, 3), name='right_camera')
right_features = xception_base(right_input)

# 合并左右视角的特征
# 包含双目视差信息，用于深度估计等任务
merged_features = layers.concatenate([left_features, right_features], axis=-1)

# 添加全局平均池化
x = layers.GlobalAveragePooling2D()(merged_features)

# 添加全连接层
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# 输出层：假设进行10类场景分类
output = layers.Dense(10, activation='softmax', name='scene_class')(x)

# 构建完整模型
stereo_model = Model(inputs=[left_input, right_input], outputs=output)

# 编译模型
stereo_model.compile(optimizer='adam',
                     loss='categorical_crossentropy',
                     metrics=['accuracy'])

print("双目视觉模型架构：")
stereo_model.summary()

# 验证模型可运行性
print("\n生成测试数据...")
left_img = np.random.randn(2, 250, 250, 3).astype('float32')
right_img = np.random.randn(2, 250, 250, 3).astype('float32')

print("执行前向推理...")
predictions = stereo_model.predict([left_img, right_img], verbose=0)
print(f"输出shape: {predictions.shape}")
print(f"Xception参数量: {xception_base.count_params()}")
print(f"完整模型参数量: {stereo_model.count_params()}")
print("验证成功：模型可以将预训练网络作为层使用")

2025-12-12 22:21:04.258638: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-12 22:21:04.265043: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-12 22:21:04.272856: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-12 22:21:04.275215: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-12 22:21:04.281183: I tensorflow/core/platform/cpu_feature_guar

I0000 00:00:1765549265.421690   35780 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549265.443368   35780 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549265.444516   35780 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1765549265.446606   35780 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

双目视觉模型架构：


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ left_camera         │ (None, 250, 250,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ right_camera        │ (None, 250, 250,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ xception            │ (None, 8, 8,      │ 20,861,480 │ left_camera[0][0… │
│ (Functional)        │ 2048)             │            │ right_camera[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 8, 8,      │          0 │ xception[0][0],   │
│ (Concatenate)       │ 4096)             │            │ xception[1][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 4096)      │          0 │ concatenate[0][0] │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │  1,048,832 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ scene_class (Dense) │ (None, 10)        │      2,570 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

...

E0000 00:00:1765549270.548656   35919 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765549270.637064   35919 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765549270.723806   35919 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


E0000 00:00:1765549271.205346   35919 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765549271.294907   35919 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765549271.383161   35919 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


E0000 00:00:1765549271.796500   35919 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.
E0000 00:00:1765549271.888482   35919 gpu_timer.cc:183] Delay kernel timed out: measured time has sub-optimal accuracy. There may be a missing warmup execution, please investigate in Nsight Systems.


I0000 00:00:1765549272.441214   35919 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


输出shape: (2, 10)
Xception参数量: 20861480
完整模型参数量: 21912882
验证成功：模型可以将预训练网络作为层使用
